In [ ]:
# script to load dicom images and save as nifty files
# by Yijun

import pydicom as dicom
import matplotlib.pyplot as plt

In [ ]:
# load corresponding dicom files, this is to check metadata
# T1w
# ds = dicom.dcmread('/hd2/research/EEG-fMRI/data/Low_field_fMRI/VOL180_1.MR.0001.0064.2021.11.19.15.19.09.841301.50903234.IMA')

# fMRI
ds = dicom.dcmread('/hd2/research/EEG-fMRI/data/Low_field_fMRI/VOL180_1.MR.0012.0001.2021.11.19.15.19.09.841301.50923783.IMA')
print(ds.pixel_array.shape)
plt.figure(figsize=(15, 15));
plt.imshow(ds.pixel_array, cmap="gray");

In [ ]:
# metadata
ds

In [ ]:
import glob

# T1w
# files = glob.glob('/hd2/research/EEG-fMRI/data/Low_field_fMRI/VOL180_1.MR.0001.0[0-9]*.2021.11.19.15.19.09.841301.[0-9]*.IMA')

# task fMRI
files = glob.glob('/hd2/research/EEG-fMRI/data/Low_field_fMRI/VOL180_1.MR.0012.00[0-9]*.2021.11.19.15.19.09.841301.[0-9]*.IMA')
files_sorted = sorted(files)
files_sorted

In [ ]:
print(len(files_sorted))

In [ ]:
# stack the fMRI dicom files to a 4D array
import numpy as np

d1_ori, d2_ori = 320, 320
d1, d2, d3, d4 = 64, 64, 25, 80
    
im_4d = np.zeros([d1, d2, d3, d4])

files_sorted = sorted(files)

for k in np.arange(80):
    print(k)
    print(files_sorted[k])
    
    ds = dicom.dcmread(files_sorted[k]);
    im = ds.pixel_array;
    im_3d = np.zeros([d1, d2, d3])
    row_start, col_start = 0, 0

    for i in np.arange(d3):
        if col_start == d2_ori:
            col_start = 0
            row_start = row_start + 64

        im_3d[:, :, i] = np.flipud(im[row_start: row_start + d1, col_start: col_start + d2]).T
        col_start = col_start + d2
        
    im_4d[:, :, :, k] = im_3d
    

In [ ]:
# stack the anatomical MRI dicom files to a 3D array
import numpy as np

d1_ori, d2_ori = 160, 160
d1, d2, d3 = 128, 160, 160
    
im_3d = np.zeros([d1, d2, d3])

files_sorted = sorted(files)

for k in np.arange(d1):
    print(k)
    print(files_sorted[k])
    
    ds = dicom.dcmread(files_sorted[k]);
    im = ds.pixel_array;
    
    im_3d[k, :, :] = np.flipud(im).T
        

In [ ]:
# save as nifty file

import nibabel as nib

# task_fmri_path = '/hd2/research/EEG-fMRI/code/low_field_exp/output/task.nii'
# nib.save(im_4d_nifti, '/hd2/research/EEG-fMRI/code/low_field_exp/output/task.nii')

im_3d_nifti = nib.Nifti1Image(im_3d, affine=np.eye(4))
nib.save(im_3d_nifti, '/hd2/research/EEG-fMRI/code/low_field_exp/output/T1.nii')